In [1]:
cd drive/My\ Drive/Notes/Spring\ 2020/NeuNet&ML/Project/Implementation/

/content/drive/My Drive/Notes/Spring 2020/NeuNet&ML/Project/Implementation


In [0]:
!python3 lstmKeras.py

Using TensorFlow backend.
2020-05-01 15:18:04.193622: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
(19720, 50, 25)
label array
[[208.]
 [207.]
 [206.]
 ...
 [  2.]
 [  1.]
 [  0.]]
2020-05-01 15:18:08.109612: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-01 15:18:08.127949: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-01 15:18:08.128706: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-05-01 15:18:08.128764: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully ope

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

data_path = "./CMAPSS_Data"
column_name =  ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
        's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
        's15', 's16', 's17', 's18', 's19', 's20', 's21' ]

In [0]:
def compute_scaled_rul_of_one_id_train(train_FD00X_of_one_id):
    max_cycle = max(train_FD00X_of_one_id['cycle'])
    rul_of_one_id = max_cycle - train_FD00X_of_one_id['cycle']
    scaled_rul_of_one_id = rul_of_one_id / max_cycle
    return scaled_rul_of_one_id.tolist()

def compute_scaled_rul_of_one_file_train(train_FD00X):
    rul = []
    for id in set(train_FD00X['engine_id']):
        rul.extend(compute_scaled_rul_of_one_id_train(train_FD00X[train_FD00X['engine_id'] == id]))
    return rul

In [0]:
def get_trainData():
  
  train_FD001 = pd.read_table(data_path+"/train_FD001.txt",header=None,delim_whitespace=True)
  train_FD002 = pd.read_table(data_path+"/train_FD002.txt",header=None,delim_whitespace=True)
  train_FD003 = pd.read_table(data_path+"/train_FD003.txt",header=None,delim_whitespace=True)
  train_FD004 = pd.read_table(data_path+"/train_FD004.txt",header=None,delim_whitespace=True)
  train_FD001.columns = column_name
  train_FD002.columns = column_name
  train_FD003.columns = column_name
  train_FD004.columns = column_name

  for data_file in ['train_FD00' + str(i) for  i in range(1,5)]:
    # have a look at the info of each data file
    eval(data_file)['RUL'] = compute_scaled_rul_of_one_file_train(eval(data_file))

  train_FD001.index = range(train_FD001.shape[0])
  train_FD002.index = range(train_FD001.shape[0],train_FD001.shape[0]+train_FD002.shape[0])
  train_FD003.index = range(train_FD001.shape[0]+train_FD002.shape[0],train_FD001.shape[0]+train_FD002.shape[0]+train_FD003.shape[0])
  train_FD004.index = range(train_FD001.shape[0]+train_FD002.shape[0]+train_FD003.shape[0],train_FD001.shape[0]+train_FD002.shape[0]+train_FD003.shape[0]+train_FD004.shape[0])

  frames = [train_FD001, train_FD002, train_FD003, train_FD004]
  train = pd.concat(frames, keys = ['FD001', 'FD002', 'FD003', 'FD004'])

  train = train.drop('engine_id', 1)
  train = train.drop('cycle', 1)
  train = train.values

  return train

In [0]:
def compute_scaled_rul_of_one_id_test(test_FD00X_of_one_id, max_cycle):    
    max_cycle = max(test_FD00X_of_one_id['cycle']) + max_cycle
    rul_of_one_id = max_cycle - test_FD00X_of_one_id['cycle']
    scaled_rul_of_one_id = rul_of_one_id / max_cycle
    return scaled_rul_of_one_id.tolist()

def compute_scaled_rul_of_one_file_test(test_FD00X,  RUL_FD00X):    
    rul = []
    for id in set(test_FD00X['engine_id']):
        rul.extend(compute_scaled_rul_of_one_id_test(test_FD00X[test_FD00X['engine_id'] == id], int(RUL_FD00X.iloc[ id - 1])))
    return rul

In [0]:
def get_testData():
  test_FD001 = pd.read_table(data_path+"/test_FD001.txt",header=None,delim_whitespace=True)
  test_FD002 = pd.read_table(data_path+"/test_FD002.txt",header=None,delim_whitespace=True)
  test_FD003 = pd.read_table(data_path+"/test_FD003.txt",header=None,delim_whitespace=True)
  test_FD004 = pd.read_table(data_path+"/test_FD004.txt",header=None,delim_whitespace=True)
  test_FD001.columns = column_name
  test_FD002.columns = column_name
  test_FD003.columns = column_name
  test_FD004.columns = column_name


  # load RUL data
  RUL_FD001 = pd.read_table(data_path+"/RUL_FD001.txt",header=None,delim_whitespace=True)
  RUL_FD002 = pd.read_table(data_path+"/RUL_FD002.txt",header=None,delim_whitespace=True)
  RUL_FD003 = pd.read_table(data_path+"/RUL_FD003.txt",header=None,delim_whitespace=True)
  RUL_FD004 = pd.read_table(data_path+"/RUL_FD004.txt",header=None,delim_whitespace=True)
  RUL_FD001.columns = ['RUL']
  RUL_FD002.columns = ['RUL']
  RUL_FD003.columns = ['RUL']
  RUL_FD004.columns = ['RUL']

  for (data_file, rul_file) in [('test_FD00' + str(i), 'RUL_FD00' + str(i)) for  i in range(1,5)]:
      # have a look at the info of each data file
      eval(data_file)['RUL'] = compute_scaled_rul_of_one_file_test(eval(data_file), eval(rul_file))

  test_FD001.index = range(test_FD001.shape[0])
  test_FD002.index = range(test_FD001.shape[0],test_FD001.shape[0]+test_FD002.shape[0])
  test_FD003.index = range(test_FD001.shape[0]+test_FD002.shape[0],test_FD001.shape[0]+test_FD002.shape[0]+test_FD003.shape[0])
  test_FD004.index = range(test_FD001.shape[0]+test_FD002.shape[0]+test_FD003.shape[0],test_FD001.shape[0]+test_FD002.shape[0]+test_FD003.shape[0]+test_FD004.shape[0])

  frames = [test_FD001, test_FD002, test_FD003, test_FD004]
  test = pd.concat(frames, keys = ['FD001', 'FD002', 'FD003', 'FD004'])

  test = test.drop('engine_id', 1)
  test = test.drop('cycle', 1)

  test = test.values

  return test